# Enterprise Deep Learning with TensorFlow: openSAP 

## SAP Innovation Center Network
```
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

### Deploying Neural Networks with TensorFlow Serving

In this notebook, we provide an end-to-end example of training, evaluating and deploying a machine learning application using TensorFlow Serving. We will train and evaluate a deep neural network using the high-level Estimator API which is also used to save the trained model. We will then export the trained model for deployment on TensorFlow Serving. Although TensorFlow Serving is a server, which is called by a client for making inference on new data, we will be simulating both the client and the server in this notebook.

#### Data
We will be using the Iris data [https://en.wikipedia.org/wiki/Iris_flower_data_set] for classifying inputs into specific flower types. More specifically, the data consists of four different attributes of a flower such as sepal width, sepal length, petal width and petal length which are used to predict the species of a flower. We will use these four attributes to build a classifier that will learn to predict the species of the flower.

#### Steps 
- Load the data from CSV files
- Construct a neural network using TensorFlow Estimator API
- Train the model using the Iris training data
- Evaluate accuracy of trained model on Iris test data
- Save model to local file system
- Export saved model to be capable of deploying on TensorFlow Serving
- Load the exported model into a new session
- Predict on new sample data

In [1]:
# Import statements
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

import numpy as np
import tensorflow as tf

print('This code requires TensorFlow v1.3+')
print('You have:', tf.__version__)

This code requires TensorFlow v1.3+
You have: 1.4.0


In [2]:
# Dataset paths
IRIS_TRAINING = "./data/iris_training.csv"
IRIS_TEST = "./data/iris_test.csv"

# URLs to download the datasets if not available locally
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

INPUT_TENSOR_NAME = 'inputs'

model_path = "./model"
model_path_serving = "./model_serving"

In [3]:
# Download the train and test data, if not found on local file system
!wget http://download.tensorflow.org/data/iris_test.csv --directory-prefix=./data/
!wget http://download.tensorflow.org/data/iris_training.csv --directory-prefix=./data/


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


#### Input functions
Input functions are used by the tf.estimator API. These function create TensorFlow operations which generate data for the model to consume. The tf.estimator.inputs.numpy_input_fn is used to produce such an input pipeline.

In [4]:
def estimator(model_path):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 30],
                                      n_classes=3,
                                      model_dir=model_path)

In [5]:
def train_input_fn():
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TRAINING,
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)()

In [6]:
def test_input_fn():
    testing_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TEST,
        target_dtype=np.int,
        features_dtype=np.float32)

    return tf.estimator.inputs.numpy_input_fn(
        x={INPUT_TENSOR_NAME: np.array(testing_set.data)},
        y=np.array(testing_set.target),
        num_epochs=1,
        shuffle=False)()

In [7]:
def serving_input_receiver_fn():
    feature_spec = {INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[4])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)()

In [8]:
classifier = estimator(model_path)

# Train model using the train input function, for 2000 steps
classifier.train(input_fn=train_input_fn, steps=2000)

# Evaluate accuracy of the trained model using the test input pipeline
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D9180FD5F8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./model\model.ckpt.
INFO:tensorflow:loss = 145.072, step = 1
INFO:tensorflow:global_step/sec: 555.158
INFO:tensorflow:loss = 10.2569, step = 101 (0.185 sec)
INFO:tensorflow:global_step/sec: 685.337
INFO:tensorflow:loss = 8.08177, step = 201 (0.145 sec)
INFO:tensorflow:global_step/sec: 746.535
INFO:tensorflow:loss = 6.2859, step = 301 

#### Exporting saved model
The Estimator API saves the model in a format that can be used to resume training or for inference. However, we might also want to create a service from the trained model that would take in requests and return results. Such a service can either be run locally on our machine, or be deployed on a server or scalably in the cloud.

In the following cells, we will export the model and run such a service locally and perform inference on new test data. 

In [10]:
# Export trained model, returns the path to the saved model
save_path = classifier.export_savedmodel(model_path_serving, serving_input_receiver_fn)

INFO:tensorflow:Restoring parameters from ./model\model.ckpt-2000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"./model_serving\\temp-b'1515016206'\\saved_model.pb"


In [11]:
# Create methods that would maintain the session using the model path as an id
sess_dict = {}
def get_session(model_id):
    global sess_dict
    config = tf.ConfigProto(allow_soft_placement=True)
    sess_dict[model_id] = tf.Session(config=config)
    return sess_dict[model_id]

def load_tf_model(model_path):
    sess = get_session(model_path)
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], model_path)
    return sess

In [12]:
# Load the exported model and restore the session
sess = load_tf_model(save_path)

INFO:tensorflow:Restoring parameters from b'./model_serving\\1515016206\\variables\\variables'


#### Debugging the restored graph
In most cases, the exported graphs would be provided as models to be served. It is hence a good practice to know operation name of the graph that would accept the input and the operation name that can be used to obtain the final output scores / probabilities.

In [13]:
# Use the restored graph's get_operations() method to know all the names of the ops available in the current graph
for op in tf.get_default_graph().get_operations():
    print (str(op.name))

global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
input_example_tensor
ParseExample/Const
ParseExample/ParseExample/names
ParseExample/ParseExample/dense_keys_0
ParseExample/ParseExample
dnn/input_from_feature_columns/input_layer/inputs/Shape
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_1
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_2
dnn/input_from_feature_columns/input_layer/inputs/strided_slice
dnn/input_from_feature_columns/input_layer/inputs/Reshape/shape/1
dnn/input_from_feature_columns/input_layer/inputs/Reshape/shape
dnn/input_from_feature_columns/input_layer/inputs/Reshape
dnn/input_from_feature_columns/input_layer/concat/concat_dim
dnn/input_from_feature_columns/input_layer/concat
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/shape
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/min
dnn/hiddenlayer_0/kernel

In [14]:
input_x_holder = sess.graph.get_operation_by_name("input_example_tensor").outputs[0]

In [15]:
predictions_holder = sess.graph.get_operation_by_name("dnn/head/predictions/probabilities").outputs[0]

In [16]:
# Predict on a new set of inputs [6.7, 2.5, 5.8, 1.8] which belongs to class 2
float_features = tf.train.Feature(float_list=tf.train.FloatList(value=[6.7, 2.5, 5.8, 1.8]))
feature_dict = {"inputs": float_features}
example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
serialized = example.SerializeToString()
score = sess.run([predictions_holder], {input_x_holder: [serialized]})

In [17]:
print(["%.4f" % score[0][0][idx] for idx in range(len(score[0][0]))])

['0.0000', '0.0002', '0.9998']


In [18]:
# Let us print the index of the largest score 
print(np.argmax(score))

2


#### Summary
We were able to train, save, export and serve a model using TensorFlow's Estimator API and Serving. Although we showed an example of serving the model on the local file system, we can use the same method to deploy to a separate machine and infer the same results by posting the test data using the IP address of that machine.